In [41]:
#----------------------------------------START----------------------------------------
# I have learned from LangchainDocs
# Source link: https://docs.langchain.com/oss/python/deepagents/long-term-memory

In [42]:
import uuid
from deepagents import create_deep_agent
from langgraph.store.memory import InMemoryStore

# Setup

In [43]:
store = InMemoryStore()
agent = create_deep_agent(
    store=store,
    use_longterm_memory=True,
    model="google_genai:gemini-2.5-flash"
)

# The /memories/ path convention

In [44]:
# Transient file (lost after thread ends)
agent.invoke({
    "messages": [{"role": "user", "content": "Write draft to /draft.txt"}]
})["messages"][-1].content

[{'type': 'text',
  'text': 'I can do that. What content would you like to include in the draft?',
  'extras': {'signature': 'CqsBAdHtim/cRa1Ol4rwmNoeGhBJQgmZrKT2H4uiudIoM/wDclpM4ejGFlXNMnVINuJJKE0Eaf90kgqKMnaTV63X0Q+auM7Baa6M/5itJ7NxwmEaboy5TlpLDSHi29jvHpz9SAsBpHkmeNnePyjG9NyPhYJCf1MaOqV7R7LgyuzT0IxmPTt+jymsK6xJPUew095nhSxL0npqpKvrgPrtZGI8Kcd06ri37NMeOp4u'}}]

In [45]:
# Persistent file (survives across threads)
agent.invoke({
    "messages":[{
        "role": "user",
        "content": "Save final report to /memories/report.txt"
    }]
})["messages"][-1].text

'There is no final report to save yet. Please tell me what you would like the report to contain.'

# Cross-thread persistence

In [47]:
# Thread 1: Write to long-term memory
config1 = {"configurable": {"thread_id": str(uuid.uuid4())}} 
agent.invoke({
    "messages":[{"role": "user", "content": "Save my preferenses to /memories/preferences.txt"}]
},
config=config1)

# Thread 2: Read from long-term memory 
config2 = {"configurable": {"thread_id": str(uuid.uuid4())}}
agent.invoke({
    "messages": [{
        "role": "user",
        "content": "What are my preferences ?"
    }]
    },
    config=config2
)
# Agent can read /memories/preferences.txt from the first thread

{'messages': [HumanMessage(content='What are my preferences ?', additional_kwargs={}, response_metadata={}, id='113d36d6-86de-4b00-8f02-ad4ec1013b65'),
  AIMessage(content=[{'type': 'text', 'text': 'I do not have access to your personal preferences. I am a large language model, trained by Google, and do not store any user-specific information.', 'extras': {'signature': 'CskBAdHtim9AZzVowwCb8F/EFzZk327Rs0A5+z9NkgFfVAgR+CaA/jNT2LENrDDn/0x5mFIzbc3EeEsVuPl6s74LWwBiAbjdbAl021e9YAswKf7MzMZmFyd1fOmz8XbJnQrIx6sh2k+m2+AluizoHTcCaOBPydtpg6eYD27KCdwmZkyXPcLAtc9B9wKjuyIQdUInSYA9VrTYHzwfnMIDya33nFJ5modYgQxDuWSxX7sdcgjHFXny4zwF7te1iFsKptqrpE5RG9tFzCIn'}}], additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--d53e8936-3f1d-4bcb-b1aa-7eb482869e5b-0', usage_metadata={'input_tokens': 4531, 'output_token

# Use cases

## User preferences

In [48]:
agent = create_deep_agent(
    store=store,
    use_longterm_memory=True,
    system_prompt="""When users tell you their preferences, save them to /memories/user_preferences.txt
    so you remember them in future conversations."""
)

## Self-improving instructions

In [49]:
agent = create_deep_agent(
    store=store,
    use_longterm_memory=True,
    system_prompt="""You have a file at /memories/instructions.txt with additonal
    instructions and preferences
    
    Read this file at the start of conversations to understand user preferences.
    
    When users provide feedback like "pleace always do X" or "I prefer X",
    update /memories/instructions.txt using the edit_file tool.""",
    model="google_genai:gemini-2.5-flash"
)

## Knowledge base

In [50]:
# Conversation 1: Learn about a project
agent.invoke({
    "messages": [{"role": "user", "content": "We're building an agent with Langchain and deepagents. Save project notes."}]
})["messages"][-1].content



"Okay, I've saved the project notes to `/memories/project_notes.txt`. What would you like to do next?"

In [51]:
# Conversation 2: Use that knowledge
agent.invoke({
    "messages": [{"role": "user", "content": "What framework are we using?"}]
})["messages"][-1].content
# Agent reads /memories/project_notes.txt from previous conversation

'I am a large language model, trained by Google. I do not use a framework in the way a software project would. I am the framework! '

## Research projects

In [52]:
research_agent = create_deep_agent(
    store=store,
    use_longterm_memory=True,
    system_prompt="""You are a research assistant.

    Save your research progress to /memories/research/:
    - /memories/research/sources.txt - List of sources found
    - /memories/research/notes.txt - Key findings and notes
    - /memories/research/report.md - Final report draft

    This allows research to continue across multiple sessions.""",
    model="google_genai:gemini-2.5-flash"
)